In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("final_dataset.csv")
df.head()

,product_id,category,unit_price,is_discountable,store_id,store_type,region,city,customer_id,gender,age,loyalty_tier,preferred_channel,transaction_id,transaction_date,channel,quantity,discount_pct
0,P0200,Home,106.31,1,S007,Outlet,Central,Grandview,C00010,M,42,Bronze,Mobile,T0042714,2025-08-01,InStore,2,0.0
1,P0140,Grocery,9.13,1,S017,Street,Central,Centrum,C00094,F,31,Bronze,InStore,T0024445,2025-08-01,InStore,1,0.0
2,P0225,Beauty,16.93,0,S038,Mall,South,Southport,C00164,M,25,Bronze,InStore,T0008413,2025-08-01,Online,2,0.0
3,P0105,Grocery,13.08,1,S039,Street,East,Rivermouth,C00197,F,24,Bronze,Online,T0037032,2025-08-01,Mobile,2,0.0
4,P0103,Home,143.33,1,S023,Mall,East,Rivermouth,C00104,M,25,Gold,InStore,T0024521,2025-08-01,InStore,1,0.0


In [3]:
df.shape

(50000, 18)

In [4]:
# Convert date to datetime
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

In [5]:
df["spend"] = df["unit_price"] * df["quantity"] * (1 - df["discount_pct"] / 100)

In [6]:
daily = (
    df.groupby(["customer_id", "transaction_date"])
      .agg(
          daily_spend=("spend", "sum"),
          total_qty=("quantity", "sum"),
          avg_price=("unit_price", "mean"),
          transactions=("spend", "count"),
          avg_discount=("discount_pct", "mean"),
          online_ratio=("channel", lambda x: (x == "Online").mean()),
          instore_ratio=("channel", lambda x: (x == "In-Store").mean())
      )
      .reset_index()
)


In [7]:
daily = daily.sort_values(["customer_id", "transaction_date"])


In [8]:
daily["next_30d_spend"] = (
    daily.groupby("customer_id")["daily_spend"]
         .transform(lambda x: x.shift(-1).rolling(window=30).sum())
)


In [9]:
daily = daily.dropna(subset=["next_30d_spend"])


In [10]:
daily.head()

,customer_id,transaction_date,daily_spend,total_qty,avg_price,transactions,avg_discount,online_ratio,instore_ratio,next_30d_spend
29,C00001,2025-09-18,593.16,2,296.58,2,0.0,0.0,0.0,5351.521955
30,C00001,2025-09-19,49.72,4,12.43,1,0.0,0.0,0.0,5318.911955
31,C00001,2025-09-21,97.61,1,97.61,1,0.0,0.0,0.0,5465.471955
32,C00001,2025-09-23,360.00,2,180.00,1,0.0,0.0,0.0,5951.431955
33,C00001,2025-09-24,500.00,2,250.00,1,0.0,0.0,0.0,6443.543630


In [11]:
daily.shape

(19211, 10)

In [15]:
static_cols = ["customer_id", "region", "city", "gender", "age", "store_type"]

static_data = (
    df.sort_values("transaction_date")
      .groupby("customer_id")
      .last()
      .reset_index()[static_cols]
)


In [17]:
daily = daily.merge(static_data, on="customer_id", how="left")


In [18]:
daily.head()

,customer_id,transaction_date,daily_spend,total_qty,avg_price,transactions,avg_discount,online_ratio,instore_ratio,next_30d_spend,region,city,gender,age,store_type
0,C00001,2025-09-18,593.16,2,296.58,2,0.0,0.0,0.0,5351.521955,South,Southport,M,41,Outlet
1,C00001,2025-09-19,49.72,4,12.43,1,0.0,0.0,0.0,5318.911955,South,Southport,M,41,Outlet
2,C00001,2025-09-21,97.61,1,97.61,1,0.0,0.0,0.0,5465.471955,South,Southport,M,41,Outlet
3,C00001,2025-09-23,360.00,2,180.00,1,0.0,0.0,0.0,5951.431955,South,Southport,M,41,Outlet
4,C00001,2025-09-24,500.00,2,250.00,1,0.0,0.0,0.0,6443.543630,South,Southport,M,41,Outlet


In [19]:
daily.shape

(19211, 15)

In [21]:
daily = daily.drop(columns=["online_ratio", "instore_ratio"])


In [23]:
daily.head()

,customer_id,transaction_date,daily_spend,total_qty,avg_price,transactions,avg_discount,next_30d_spend,region,city,gender,age,store_type
0,C00001,2025-09-18,593.16,2,296.58,2,0.0,5351.521955,South,Southport,M,41,Outlet
1,C00001,2025-09-19,49.72,4,12.43,1,0.0,5318.911955,South,Southport,M,41,Outlet
2,C00001,2025-09-21,97.61,1,97.61,1,0.0,5465.471955,South,Southport,M,41,Outlet
3,C00001,2025-09-23,360.00,2,180.00,1,0.0,5951.431955,South,Southport,M,41,Outlet
4,C00001,2025-09-24,500.00,2,250.00,1,0.0,6443.543630,South,Southport,M,41,Outlet


In [22]:
daily.shape

(19211, 13)

In [24]:
daily.to_csv("short_term_spend_model_data.csv", index=False)
